# DR CRC Calibration

Author: Sophie Wagner, sw3767@cumc.columbia.edu

In [1]:
# Required Packages
import numpy as np  # For matrix manipulation
import pandas as pd  # For output/input data processing
import matplotlib.pyplot as plt  # For visualizations
from csaps import csaps
from scipy.interpolate import interp1d
from tqdm import tqdm
from datetime import datetime

# Add the src directory to the Python path
import sys
import os
sys.path.append(os.path.abspath('../src'))

# Load .py files
import common_functions as func
import calibration_plots as p
import configs as c
import gof


# Some aesthetic options
np.set_printoptions(suppress=True, linewidth=300, formatter={'float': '{: 0.9f}'.format})
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Matrix setup, normalization, constraints

In [2]:
def row_normalize(matrix):
    for age_layer in range(matrix.shape[0]):  # Loop over each age layer
        layer = matrix[age_layer]
        # Calculate the sum of non-diagonal elements for each row
        sum_of_columns = np.sum(layer, axis=1) - np.diag(layer)
        # Set the diagonal elements
        np.fill_diagonal(layer, 1 - sum_of_columns)
    return matrix


def create_matrix():
    matrix = np.zeros((len(c.age_layers), len(c.health_states), len(c.health_states)))
    matrix[:, 0, 1] = func.probtoprob(0.005)  # Healthy to LR
    matrix[:, 1, 2] = func.probtoprob(0.015)  # LR to HR
    matrix[:, 2, 3] = func.probtoprob(0.05)  # HR to uLoc
    matrix[:, 3, 4] = func.probtoprob(0.45)  # uLoc to uReg
    matrix[:, 4, 5] = func.probtoprob(0.50)  # uReg to uDis
    matrix[:, 3, 6] = func.probtoprob(0.20)  # uLoc to dLoc
    matrix[:, 4, 7] = func.probtoprob(0.60)  # uReg to dReg
    matrix[:, 5, 8] = func.probtoprob(0.90)  # uDis to dDis

    matrix = add_acm(matrix)  # ACM
    matrix = add_csd(matrix)  # CSD
    matrix = constrain_matrix(matrix)  # constrain
    matrix = row_normalize(matrix)  # normalize

    return matrix

locreg=func.probtoprob(0.1)
def constrain_matrix(matrix):
    matrix = np.clip(matrix, 0.0, 0.5)

    # Progression Block
    matrix[:, 0, 1] = np.maximum(0.000001, matrix[:, 0, 1])  # not below 0
    matrix[:, 1, 2] = np.maximum(matrix[:, 0, 1], matrix[:, 1, 2])  
    matrix[:, 2, 3] = np.maximum(matrix[:, 1, 2], matrix[:, 2, 3])
    matrix[:, 3, 4] = np.maximum(locreg, matrix[:, 3, 4])
    matrix[:, 3, 4] = np.maximum(matrix[:, 2, 3], matrix[:, 3, 4])
    matrix[:, 4, 5] = np.maximum(matrix[:, 3, 4], matrix[:, 4, 5])

    # Detection Block
    matrix[:, 3, 6] = np.maximum(0.000001, matrix[:, 3, 6])
    matrix[:, 4, 7] = np.maximum(matrix[:, 3, 6], matrix[:, 4, 7])
    matrix[:, 5, 8] = np.maximum(matrix[:, 4, 7], matrix[:, 5, 8])
    
    # Age dependencies
    matrix[:, 0, 1] = np.maximum.accumulate(matrix[:,0,1])
    # matrix[:, 1, 2] = np.maximum.accumulate(matrix[:,0,1])
    # matrix[:, 2, 3] = np.maximum.accumulate(matrix[:,0,1])

    return matrix


def add_acm(matrix):
    matrix[:, 0, 10] = c.acm_rate  # Healthy to ACM
    matrix[:, 1:3, 12] = c.acm_rate[:, np.newaxis]  # Polyp to ACM
    matrix[:, 3:6, 13] = c.acm_rate[:, np.newaxis]  # Undiagnosed to ACM
    matrix[:, 6:9, 11] = c.acm_rate[:, np.newaxis]  # Cancer to ACM
    matrix[:, 9, 9] = 1  # Stay in CSD
    matrix[:, 10, 10] = 1  # Stay in ACM
    matrix[:, 11, 11] = 1  # Stay in Cancer ACM
    matrix[:, 12, 12] = 1  # Stay in Polyp ACM
    matrix[:, 13, 13] = 1  # Stay in uCRC ACM

    return matrix


def add_csd(matrix):
    matrix[:, 6, 9] = c.csd_rate[:, 0]
    matrix[:, 7, 9] = c.csd_rate[:, 1]
    matrix[:, 8, 9] = c.csd_rate[:, 2]
    return matrix

### Markov model

In [3]:
def run_markov(matrix, starting_age=20, max_age=84):
    
    current_age = starting_age
    stage, age_layer = 1, 0
    month_pop, pop_log = c.starting_pop, c.starting_pop
    inc_log = np.zeros(pop_log.shape)  # to track new incidences in each state
    matrixT = matrix.transpose(0,2,1)
    inflow_matrix = np.tril(matrixT, k=-1)
    
    while current_age <= max_age:
        
        # Matrix multiplication (state transition)
        mat, inflow_mat = matrixT[age_layer], inflow_matrix[age_layer] 
        month_inc = np.matmul(inflow_mat, month_pop)  # (9, 9)(9, 1)->(9, 1)
        month_pop = np.matmul(mat, month_pop)  # (9, 9)(9, 1)->(9, 1)
        
        # Add to log
        inc_log = np.concatenate((inc_log, month_inc), axis=1)
        pop_log = np.concatenate((pop_log, month_pop), axis=1)
        
        stage += 1
        if stage % 12 == 0:
            current_age += 1
            if current_age in c.ages_5y: 
                age_layer = min(age_layer+1, 12)

    incidence = inc_log.copy()  # make (14,780)
    dead_factor = np.divide(c.N, c.N - pop_log[9:, :].sum(axis=0))  # inc and prev denominator is out of living only
    prevalence = np.zeros(pop_log.shape)  # (14,65)

    for state in range(14):
        incidence[state, :] = np.multiply(incidence[state, :], dead_factor)
        prevalence[state, :] = np.multiply(pop_log[state, :], dead_factor)

    incidence = incidence.reshape(len(c.health_states), 65, 12).sum(axis=2)  # getting annual incidence (rate per 100k)
    incidence_unadj = inc_log.reshape(len(c.health_states), 65, 12).sum(axis=2)  # getting inc unadjusted
    prevalence = prevalence.reshape(len(c.health_states), 65, 12).mean(axis=2)  # getting mean annual prevalence
    
    return incidence, prevalence, incidence_unadj, pop_log

### Simulated annealing

In [4]:
age_mids = [22.5,27.5,32.5,37.5,42.5,47.5,52.5,57.5,62.5,67.5,72.5,77.5,82.5]
def step(matrix, step_size, num_adj=5):
    new_matrix = np.copy(matrix)
    step_mat = np.random.choice(len(c.points), size=num_adj, replace=True)
    step_age = np.random.choice(len(c.age_layers_5y), size=num_adj, replace=True)

    for i in range(num_adj):
        (from_state, to_state) = c.points[step_mat[i]]
        step_param = np.mean(new_matrix[:, from_state, to_state]) * step_size
        new_matrix[step_age[i], from_state, to_state] += np.random.uniform(low=-step_param, high=step_param)
    
    new_matrix[12,:,:] = np.minimum(new_matrix[11,:],new_matrix[12,:,:])  # Limit potential increase before splining
    new_matrix = csaps(age_mids, new_matrix, smooth=0.01, axis=0)(age_mids).clip(0.0,1.0)
    
    new_matrix = constrain_matrix(new_matrix)
    new_matrix = add_acm(new_matrix)
    new_matrix = add_csd(new_matrix)
    new_matrix = row_normalize(new_matrix)

    return new_matrix

In [5]:
def progress_report(iteration, best_eval, best_log, ticker, best_t):
    """
    Prints progress report during simulated annealing.
    """
    log_adj, _, inc_log, _ = best_log
    total_dxd = np.sum(inc_log[6:9, :]) / c.N
    total_pol = np.sum(inc_log[12, :]) / c.N 
    print(f"{iteration}: Best Eval: {best_eval:.5f}, CRC: {total_dxd:.5f}, Polyp: {total_pol:.5f}, Tick: {ticker}")

    if iteration % 50000 == 0:
        transition_probs = p.extract_transition_probs(best_t, c.health_states, c.desired_transitions)
        print(f"Detailed Progress Report, Iteration = {iteration}")
        p.print_trans_probs(transition_probs)

In [6]:
def simulated_annealing(n_iterations, step_size, start_tmat=None, n_adj=7, verbose=False, starting_temp=1, print_interval=2500, obj=""):
    """
    Performs simulated annealing to optimize a transition matrix.

    Args:
        n_iterations (int): Number of iterations for optimization.
        step_size (float): Step size for parameter adjustments.
        start_tmat (numpy.ndarray): Initial transition matrix.
        n_adj (int): Number of parameters to adjust per step.
        starting_temp (float): Initial temperature for annealing.
        verbose (bool): Whether to print progress reports.
        print_interval (int): Interval for progress reporting.

    Returns:
        numpy.ndarray: Optimized transition matrix.
    """
    best_t = np.copy(start_tmat)
    best_log = run_markov(best_t)
    best_eval = gof.objective(run_markov(start_tmat), -1, obj)
    curr_t, curr_eval = best_t, best_eval
    ticker = 0

    with tqdm(total=n_iterations, desc="Simulated annealing progress", unit="iteration") as pbar:
        
        for i in range(n_iterations):

            # Run model
            candidate_t = np.copy(curr_t)
            candidate_t = step(candidate_t, step_size, n_adj)
            candidate_log = run_markov(candidate_t)
            candidate_eval = gof.objective(candidate_log, i, obj)  # Evaluate candidate point

            # Update "best" if better than candidate
            if candidate_eval < best_eval:
                ticker = 0
                best_t, best_eval = np.copy(candidate_t), np.copy(candidate_eval)
                best_log = run_markov(best_t)

            else:
                ticker += 1

            # Calculate temperature and Metropolis acceptance criterion
            t = starting_temp / (1 + np.log(i + 1))
            diff = candidate_eval - curr_eval
            metropolis = np.exp(-diff / t)

            if diff < 0 or np.random.random() < metropolis:
                curr_t, curr_eval = np.copy(candidate_t), candidate_eval

            # Print progress report
            if verbose and i > 0 and i % print_interval == 0:
                progress_report(i, best_eval, best_log, ticker, best_t)

            # Check if we should update "curr"
            diff = (candidate_eval - curr_eval)  # difference between candidate and current point evaluation
            metropolis = np.exp(-diff / t)  # calculate metropolis acceptance criterion
            if (diff < 0 or np.random.random() < metropolis):  # check if we should keep the new point
                curr_t, curr_eval = np.copy(candidate_t), np.copy(candidate_eval)  # store the new current point
                ticker = 0

            pbar.update(1)

    print("Final score: ", best_eval)
    
    return best_t

In [7]:
def run_sa(tmat=None, save_all=False, n_iterations=50000, step_size=0.2, n_adj=5, starting_temp=1, obj=""):
    
    start_tmat = None
    start_tmat = tmat if tmat is not None else create_matrix()   
    initial_score = gof.objective(run_markov(start_tmat), -1, obj)
    print(f"Initial score: {round(initial_score, 5)}")
    # print("Starting calibration...")
    
    result = simulated_annealing(n_iterations=n_iterations, step_size=step_size, start_tmat=tmat, n_adj=n_adj, starting_temp=starting_temp, verbose=True, obj=obj)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    
    curr_tmat = result.copy()
    curr_log = run_markov(curr_tmat)

    # Extract transition probabilities
    transition_probs = p.extract_transition_probs(curr_tmat, c.health_states, c.desired_transitions)

    # Saving
    if save_all:
        # Save the with the timestamp in the filenames
        tmat_path, plot_path, probs_path = c.OUTPUT_PATHS["tmats"], c.OUTPUT_PATHS["plots"], c.OUTPUT_PATHS["probs"]
        np.save(f"{tmat_path}/{timestamp}_tmat.npy", curr_tmat)

        p.print_trans_probs(transition_probs, save_imgs=True, outpath=plot_path, timestamp=timestamp)
        p.print_trans_probs(transition_probs)
        p.plot_tps(curr_tmat, save_imgs=True, outpath=plot_path, timestamp=timestamp)
        p.plot_vs_seer(curr_log, c.seer_inc, save_imgs=True, outpath=plot_path, timestamp=timestamp)
        p.plot_vs_seer_total(curr_log, c.seer_inc, save_imgs=True, outpath=plot_path, timestamp=timestamp)
        
        out = np.zeros((len(c.points), 80))
        for idx, (from_state, to_state) in enumerate(c.points):
            out[idx] = curr_tmat[:, from_state, to_state]

        pd.DataFrame(out).to_csv(f"{probs_path}/{timestamp}_tps.csv")

    # else:
        # p.print_trans_probs(transition_probs)
        # p.plot_tps(curr_tmat)
        # p.plot_vs_seer(curr_log, c.seer_inc)
        # p.plot_vs_seer_total(curr_log, c.seer_inc)

    return curr_tmat

## Run SA

### Fold

In [8]:
result = create_matrix()
result = run_sa(result, True, 1000000, step_size=0.3, n_adj=7, obj="pol", starting_temp=25)

Initial score: 481870.91339


Simulated annealing progress:   0%|          | 0/1000000 [00:00<?, ?iteration/s]C:\Users\sophi\AppData\Local\Temp\ipykernel_46420\3407900094.py:45: RuntimeWarning: overflow encountered in exp
  metropolis = np.exp(-diff / t)
Simulated annealing progress:   0%|          | 2505/1000000 [00:38<10:15:19, 27.02iteration/s]

2500: Best Eval: 1494.13909, CRC: 0.05011, Polyp: 0.18064, Tick: 745


Simulated annealing progress:   1%|          | 5013/1000000 [01:22<3:10:30, 87.05iteration/s] 

5000: Best Eval: 1388.75518, CRC: 0.04940, Polyp: 0.18143, Tick: 122


...

892500: Best Eval: 880.18858, CRC: 0.04906, Polyp: 0.18110, Tick: 625475


Simulated annealing progress:  90%|████████▉ | 895011/1000000 [46:56:05<30:56, 56.57iteration/s]  

895000: Best Eval: 880.18858, CRC: 0.04906, Polyp: 0.18110, Tick: 627975


Simulated annealing progress:  90%|████████▉ | 895023/1000000 [46:56:05<5:30:17,  5.30iteration/s]


KeyboardInterrupt: 

### Current

## Post-processing

In [54]:

def objective_cp(tmat):
    score = 0
    cp = cancer_progression(tmat)
    score += np.square(cp-3.0).sum()
    return score
    
def cancer_progression(tmat):
    """
    Calculate time from preclinical local to preclinical distant. MFPT
    """
    p_12 = tmat[:, 3, 4] # loc to reg
    p_23 = tmat[:, 4, 5] # reg to dis
    p_11 = tmat[:, 3, 3] # stay loc
    p_22 = tmat[:, 4, 4] # stay reg
    p_33 = tmat[:, 5, 5] # stay dis
    
    cp = (1 + p_12 * (1 + p_23 * (1 / (1 - p_33))) * (1 / (1 - p_22))) * (1 / (1 - p_11))
    
    return cp
    
    
def sojourn_time_weighted(tm, metric="mean"):
    """
    Calculate  time spent in each path.
    """
    in_loc, in_reg, in_dis = [1/(1-tm[:, x, x]) for x in [3,4,5]]
    mloc = in_loc
    mreg = in_loc + in_reg
    mdis = (in_loc + in_reg * tm[:, 3, 4]) + in_dis
    
    if metric == "mean": # Mean across paths per age 
        sj_time =  np.mean([mloc, mreg, mdis], axis=0)
    else: # Each path per age
        sj_time = np.array([mloc, mreg, mdis])
    
    return sj_time

def sojourn_time_weighted2(tm):
    """
    Calculate  time spent in each path, weighted by stage.
    """
    in_loc, in_reg, in_dis = [1/(1-tm[:, x, x]) for x in [3,4,5]]
    mloc = in_loc
    mreg = in_loc * tm[:, 3, 4] + in_reg
    mdis = in_loc * tm[:, 3, 4] + in_reg * tm[:, 4, 5] + in_dis
    sj_time = np.array([mloc, mreg, mdis])
    
    return sj_time

def sojourn_time_in_stage(tmat):
    """
    Calculate mean time in each state. Average over all states.
    """
    sojourn_times = np.zeros((3,80))
    for i in np.arange(3,6,1):
        p_stay = tmat[:, i, i]
        sojourn_times[i-3] = 1 / (1 - p_stay)
    return sojourn_times

In [5]:
from scipy.stats import sem, t

def summarize_data(data):
    """
    Returns the min, max, median, mean, and 95% confidence interval of a dataset.
    
    Parameters:
        data (list or numpy array): Input data
    
    Returns:
        dict: Summary statistics
    """
    if len(data) == 0:
        return "Data is empty."
    
    # Convert to numpy array for convenience
    data = np.array(data)
    
    # Summary statistics
    min_val = np.min(data)
    max_val = np.max(data)
    median_val = np.median(data)
    mean_val = np.mean(data)
    
    # Compute 95% confidence interval
    confidence = 0.95
    n = len(data)
    if n > 1:
        std_err = sem(data)
        h = std_err * t.ppf((1 + confidence) / 2, n - 1)
        ci_lower = mean_val - h
        ci_upper = mean_val + h
    else:
        ci_lower = ci_upper = mean_val  # No confidence interval for a single data point

    return {
        "min": min_val,
        "max": max_val,
        "median": median_val,
        "mean": mean_val,
        "95% CI": (ci_lower, ci_upper)
    }

In [ ]:
loc, reg, dis = sojourn_time_weighted(tmat, metric="")
total = sojourn_time_weighted(tmat)
print(summarize_data(loc))
print(summarize_data(reg))
print(summarize_data(dis))
print(summarize_data(total))

plt.plot(np.arange(0,80,1), loc, color="blue", label="L")
plt.plot(np.arange(0,80,1), reg, color="red", label="R")
plt.plot(np.arange(0,80,1), dis, color="green", label="D")
plt.plot(np.arange(0,80,1), total, color="grey", label="All")
plt.title("Sojourn Time by Age (cumulative uL->dX)")
plt.xlabel("Age")
plt.ylabel("Months")
plt.legend()
plt.show()

import seaborn as sns
plt.hist(loc, bins=30, density=True, alpha=0.6, color="blue", label="Loc")
plt.hist(reg, bins=30, density=True, alpha=0.6, color="red", label="Reg")
plt.hist(dis, bins=30, density=True, alpha=0.6, color="green", label="Dis ")
sns.kdeplot(loc, fill=True, color="blue", alpha=0.3, clip=(0, None))  
sns.kdeplot(reg, fill=True, color="red", alpha=0.3, clip=(0, None))  
sns.kdeplot(dis, fill=True, color="green", alpha=0.3, clip=(0, None))
plt.title("Density Distribution of Sojourn Time by Stage at DX (cumulative uL->dX)")
plt.xlabel("Months")
plt.ylabel("Density")
plt.legend()
plt.show()

## Extend Parameters to Age 100

- Use cubic spline interpolation to interpolate probabilities from 20-100
- Anchor increasing parameters at age 100 to prevent exponential incidence increase

In [ ]:
def smooth_tmat(mat, save=False, outpath="", timestamp=""):
    
    # Interpolation without anchoring
    tmat = csaps(age_mids, mat, smooth=0.01, axis=2)(np.linspace(20,100,81)) 
    
    # Anchor with mean at 100 (if mean < last age bucket value) and Smooth interpolation over age
    tmat_anchored = np.concatenate([mat, np.minimum(mat[-1:, :, :], np.mean(mat, axis=0, keepdims=True))], axis=0)
    tmat_anchored = csaps([25,35,45,52.5,57.5,62.5,67.5,72.5,77.5,82.5, 100], tmat_anchored, smooth=0.01, axis=2)(np.linspace(18,100,83)).clip(0.0, 1.0)

    # If smoothed tmat is exp increasing towards end, used anchored probs
    increasing_at_100 = tmat[-2:-1, :, :] < tmat[-1:, :, :]
    tmat_anchored = np.where(increasing_at_100, tmat_anchored, tmat)
    transition_mask = np.zeros((14, 14), dtype=bool)
    from_states, to_states = zip(*c.points)
    transition_mask[from_states, to_states] = True
    tmat_anchored = np.where(increasing_at_100 & transition_mask[np.newaxis, ...], tmat_anchored, tmat)

    tmat = tmat_anchored
    
    # HANDLE NEGATIVE CASE: Sometimes spline dips below 0 and becomes positive again, causing a spike after clipping the matrix to 0 and 1.
    # Find the index of the first negative before a positive index from the last age
    neg_after_pos = ((tmat[:, :, :-1, :, :] >= 0) & (tmat[:, :, 1:, :, :] < 0)) # any negative after positive marked True
    neg_after_pos = np.insert(neg_after_pos, 0, False, axis=0) # insert False at the first index to make arr shape same as tmat, by definition first idx is False
    neg_after_pos_reversed = np.flip(neg_after_pos, axis=0)
    last_neg_after_pos_til_end = np.flip(np.where(np.cumsum(neg_after_pos_reversed, axis=0) == 0, True, neg_after_pos_reversed), axis=0) # reverse to find the last True, set values from last True until end to True
    mask = np.any(neg_after_pos, axis=2).reshape(2, 2, 1, 18, 18) & last_neg_after_pos_til_end # only apply this to indices that have negative at the end
    # Set indices from last negative after positive to end to zero
    tmat[mask] = 0

    tmat = row_normalize(tmat)

    if save:
      os.makedirs(outpath, exist_ok=True)
      # Save tmat
      np.save(f"{outpath}/{timestamp}_100.npy", tmat)
    return tmat